In [7]:
from llm_lasso.task_specific_lasso.llm_lasso import run_repeated_llm_lasso_cv
from llm_lasso.task_specific_lasso.plotting import plot_heatmap
from llm_lasso.data_splits import read_train_test_splits, save_train_test_splits, read_baseline_splits
from llm_lasso.baselines.data_driven import run_all_baselines_for_splits
import pandas as pd
import pickle
from llm_lasso.utils.small_scale_data import *
import numpy as np
import warnings
import json
warnings.filterwarnings("ignore")  # Suppress warnings

In [4]:
%load_ext autoreload
%autoreload 2

## Spotify

In [ ]:
# Save train and text splits
X, y = process_spotify_csv()
save_train_test_splits(X, y, "data/train_test_splits/Spotify", 10, seed=0)

In [5]:
# Read the splits back in
(x_train, x_test, y_train, y_test) = read_train_test_splits("data/train_test_splits/Spotify", 10)

In [ ]:
# Run baselines
run_all_baselines_for_splits(
    x_train, y_train, "data/baselines/Spotify", min=0, max=len(x_train[0].columns) + 1, step=len(x_train[0].columns))

In [7]:
# Make sure you have already gotten LLM-Lasso scores from our/llasso_scores.py
# and LLM-Score scores from baselines/llm_select.py

In [ ]:
scores_rag = np.array(
    np.load("data/scores/Diabetes/gene_scores_RAG.pkl", allow_pickle=True)
)
scores_plain = np.array(
    np.load("data/scores/Diabetes/gene_scores_plain.pkl", allow_pickle=True)
)

importance_scores_list={
    "rag": scores_rag,
    "plain": scores_plain,
}
model_names = importance_scores_list.keys()
methods = [f"1/imp - {name}" for name in model_names] + ["Lasso"]

In [ ]:
feature_baseline = read_baseline_splits("data/baselines/Spotify", n_splits=10, key=f"{len(x_train[0].columns)}")

with open("data/scores/Diabetes/llmselect_selected_genes.json", "r") as f:
    llm_select_genes = json.load(f)[f"{len(x_train[0].columns)}"]

feature_baseline["llm_score"] = [llm_select_genes] * 10

baseline_names = feature_baseline.keys()             

In [ ]:
(res, _) = run_repeated_llm_lasso_cv(
    x_train_splits=x_train, x_test_splits=x_test,
    y_train_splits=y_train, y_test_splits=y_test,
    importance_scores_list=importance_scores_list,
    feature_baseline=feature_baseline,
    baseline_names=baseline_names,
    model_names=model_names,
    methods=methods,
    n_splits=10,
    folds_cv=10,
    score_type = "pf",
    lambda_min_ratio=0.001,
    n_threads=8,
    plot_error_bars=False,
    bolded_methods=["1/imp - rag"],
)